In [721]:
import math
import pandas as pd
import itertools

In [722]:
def euklides_measure(vec1, vec2):
    result = 0
    # ewentualnie mozna liczyc odleglosci euklidesowe dla wektorow o roznej odl
    if len(vec1) != len(vec2):
        return 0
    for i in range(len(vec1)):
        result = result + (vec1[i]-vec2[i])**2
    result = math.sqrt(result)

    #korekta, gdy odleglosc to 0, zapisujemy 0 w inny sposób
    if result == 0:
        result = 1e-21

    return result

In [723]:
def get_csv(x):

    df = pd.read_csv(x)
    leng = len(df.columns)
    return df, leng

In [724]:
def write_to_sim_vec(sim_vec, index, emeasure, vec_of_vecs):
    sim_vec[index] = emeasure
    vec_of_vecs.append(sim_vec)

    return vec_of_vecs


In [725]:
def compare(test, which_line, agent, a_index, dec_atr):
    test = test_file.iloc[which_line]

    #zliczac do zmiennej liczbe wartosci atrybutu decyzyjnego -> podawana w zmiennej
    vec_of_vecs = []

    train_vec = [None] * (a_index - 1)
    test_vec = [None] * (a_index - 1)

    #tworzenie wektora odległości bezwzględnej
    for index, row in agent.iterrows():
        sim_vec = [0] * len(dec_atr)

        for i in range (len(train_vec)):
            train_vec[i] = row[agent.columns[i]]
            test_vec[i] = test[agent.columns[i]]

        emeasure = euklides_measure(train_vec,test_vec)

        #w innej funkcji wybierac wszystkie odpowiadajace dla wybranego k

        for atr in dec_atr:

            if row[agent.columns[a_index-1]] == atr:
                vec_of_vecs = write_to_sim_vec(sim_vec, dec_atr.index(atr), emeasure, vec_of_vecs)


    return vec_of_vecs


In [726]:
def find_dec_atr(files):
    dec_atr = []
    for file in files:
        [agent, a_index] = get_csv(file)
        for index, row in agent.iterrows():
            dec = row[agent.columns[a_index-1]]
            if dec not in dec_atr:
                dec_atr.append(dec)

    return dec_atr

In [727]:
def make_vector(files, test_file, which_line, num_of_dec):
    all_vecs = []
    for file in files:
        [agent, a_index] = get_csv(file)
        all_vecs.append(compare(test_file, which_line, agent, a_index, num_of_dec))
    return all_vecs



In [728]:
def find_the_closest(k, list, dec_atr, agent_num):

    final_vec = [[[0 for x in range(len(dec_atr))] for x in range(k)] for x in range(agent_num)]
    ixs_to_delete = [[0 for x in range(len(dec_atr))] for x in range(agent_num)]

    save = False

    #[[[0,0,0..],[..],[..],..]],[[0,0,0..],[..],[..],..]],[[0,0,0..],[..],[..],..]],..]
    for i in range(k):
        for j in range(len(list)):
            for l in range(len(list[j])):
                for m in range(len(list[j][l])):
                    if list[j][l][m] != 0:
                        final_vec, save = try_val(final_vec, j, i, list[j][l], m)
                        if save:
                            ixs_to_delete[j][m] = l
    # deletion:
    # sorting the inner list to avoid moving important index down

        for i in range(len(ixs_to_delete)):
            for j in sorted(ixs_to_delete[i], reverse= True):
                del list[i][j]

    # jesli k-ty obiekt ma taka sama odl jak inne obiekty, to wszystkie obiekty sa brane pod uwage
    # 0.1, 0.2, 0.2, 0.3 -> dla k = 2 bierzemy 3 pierwsze obiekty, bo mają tą samą wartość

    return final_vec

In [729]:
def try_val(final_vec, agent_in_final, k_in_final, vec, m):

    fin = final_vec[agent_in_final][k_in_final]
    save = False

    for i in range(len(fin)):

        if i == m:
            if fin[m] == 0:
                fin[m] = vec[m]
                save = True

            elif fin[m] > vec[m]:
                fin[m] = vec[m]
                save = True

    return final_vec, save


In [730]:
def change_to_relative(k_closest):
    for k in k_closest:
        for l in k:
            sum = 0
            #zliczamy sume
            for m in l:
                sum += m
            #obliczamy wartosci wzgledem sumy i odejmujemy
            # od 1 aby nie byly najmniejsze, tylko najwieksze, aby utworzyc podobienstwo
            for i in range(len(l)):
                l[i] = 1 - l[i]/sum
            #podobniestwo wzgledne (udzial procentowy)
            sum = 0
            for m in l:
                sum += m
            for i in range(len(l)):
                l[i] = l[i]/sum

    # to na koncu (normalizacja)

    return k_closest

In [731]:
def unify_k_vecs(relative_vecs, agent_num):
    unified_vec = [[0 for x in range(len(relative_vecs[0][0]))]for x in range(agent_num)]
    for i in range(len(relative_vecs)):
        for j in range(len(relative_vecs[i])):

            for k in range(len(relative_vecs[i][j])):
                unified_vec[i][k] += relative_vecs[i][j][k]

        for k in range(len(relative_vecs[i][j])):
            unified_vec[i][k] /= len(relative_vecs[i])

    # tu nie normalizować, tylko raz na końcu

    return unified_vec


In [732]:
def sub_lists (l):
    lists = [[]]
    for i in range(len(l) + 1):
        for j in range(i):
            lists.append(l[j: i])
    return lists

In [733]:
#q od 0 do 1 - wartosc procentowa, po osiagnieciu ktorej decyzja jest przelosowana
def Shapley_Shubik(vec, q, dec_atr):

    force_list = [0 for x in range(len(vec))]

    #na początku szukamy list dla poszczególnych decyzji, zapisujemy ją (listę) jako temp_vec
    for i in range(len(dec_atr)):

        #temp_vec jest pomocniczy, aby obliczyc ostateczną siłę agenta: [dec1a1, dec1a2, dec1a3..]
        temp_vec = [0 for x in range(len(vec))]

        #uzupelnienie wektora
        for m in range(len(vec)):
            temp_vec[m] = vec[m][i]

        #wyciągamy pierszy element tabeli i później odkładamy go na górę (koniec)
        agent_sums = []
        for l in range(len(temp_vec)):

            # print("temp_vec: ",temp_vec)
            x = temp_vec.pop(i)
            subsets = sub_lists(temp_vec)
            sum_to_Shapley = 0
            #sprawdzamy gdzie x ma decydujace znaczenie
            for subs in subsets:
                sum = 0
                point = False
                #sprawdzamy, czy x jest samo większe lub równe wymaganej liczbie
                if len(subs) == 0:
                    if x >= q:
                        point = True
                #sprawdzamy czy x w połączeniu z innymi wartościami jest decydujące
                else:
                    for j in range(len(subs)):
                        sum += subs[j]
                        if j == len(subs) - 1:
                            if sum >= q:
                                pass
                            elif sum + x >= q:
                                point = True
                #jeżeli x jest decydujące, dodajemy do sum_to_Shapley
                if point:
                    sum_to_Shapley += 1

            temp_vec.append(x)

            print("agent nr ",l+1, " ma sile ",sum_to_Shapley/len(subsets))

            agent_sums.append(sum_to_Shapley/len(subsets))
        print(agent_sums)
        #po sprawdzeniu, dodajemy ten element na koniec, teraz 1 jest 2 itd
        #aby potem sprawdzić następny numer a nie znow ten sam

    return force_list

    # algorytm poprawić bo musi być po agentach a nie po decyzjach!!!


In [734]:
def count_final_vec(unified_vec, f_list):

    help_vec = [[0 for x in range(len(unified_vec[0]))]for x in range(len(unified_vec))]
    final_vec = [0 for x in range(len(unified_vec[0]))]

    for i in range(len(unified_vec)):
        for j in range(len(unified_vec[i])):
            help_vec[i][j] = unified_vec[i][j] * f_list[i][j]

    for i in range(len(help_vec)):
        for j in range(len(help_vec[i])):
            final_vec[j] += help_vec[i][j]

    return final_vec

In [735]:
files = ['d1.csv', 'd2.csv', 'd3.csv', 'd4.csv', 'd5.csv', 'd6.csv', 'd7.csv']
test_file, t_index = get_csv('test_w.csv')
dec_atr = find_dec_atr(files)
ageent_num = len(files)

# for i in range(len(test_file)):

line_to_compare = 0

list = make_vector(files, test_file, line_to_compare, dec_atr)

k_closest = find_the_closest(3, list, dec_atr, ageent_num)

print("k_closest: ", k_closest)

relative_vecs = change_to_relative(k_closest)

print("relative_vecs: ",relative_vecs)

unified_vec = unify_k_vecs(relative_vecs, ageent_num)

print("unified_vec: ",unified_vec)

f_list = Shapley_Shubik(unified_vec, 0.5, dec_atr)

print("f_list: ",f_list)

# final_vec = count_final_vec(unified_vec,f_list)
#
# print("final_vec: ",final_vec)

print('decyzja agentow: ', dec_atr[final_vec.index(max(final_vec))])
print('decyzja poprawna: ', test_file.iloc[line_to_compare][t_index - 1])
print('Czy dobra decyzja?: ', dec_atr[final_vec.index(max(final_vec))] == test_file.iloc[line_to_compare][t_index - 1], end='\n\n')

    # 160/94  -> T/F
    # do poprawy unifikacja agentów -> nie robić jej
    # do dodania składowanie wyników w jakimś słowniku..
    # siła ma być wartością a nie wektorem (dodać wartości sił)

k_closest:  [[[12.165525060596439, 3.1622776601683795, 36.796738985948195, 1.4142135623730951], [12.206555615733702, 3.605551275463989, 38.535697735995385, 3.0], [13.45362404707371, 4.47213595499958, 39.68626966596886, 4.0]], [[13.0, 2.449489742783178, 37.603191353926334, 1e-21], [13.038404810405298, 3.0, 39.06404996924922, 2.449489742783178], [14.66287829861518, 3.7416573867739413, 39.408120990476064, 3.0]], [[5.196152422706632, 2.23606797749979, 45.16635916254486, 1.7320508075688772], [7.280109889280518, 2.449489742783178, 45.17742799230607, 1.7320508075688772], [10.099504938362077, 2.8284271247461903, 46.10856753359401, 2.23606797749979]], [[13.638181696985855, 9.38083151964686, 250.28983199482954, 3.0], [16.0312195418814, 10.677078252031311, 252.48168250389966, 4.242640687119285], [31.416556144810016, 11.045361017187261, 258.53046242174247, 5.0]], [[20.273134932713294, 12.96148139681572, 252.69942619642015, 6.082762530298219], [31.464265445104548, 16.792855623746664, 257.3266406729